In [1]:
# 구글드라이브 mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [40]:
cd /content/drive/MyDrive/MLDL_telcobiz_embedding_classifier

/content/drive/MyDrive/MLDL_telcobiz_embedding_classifier


In [3]:
# 형태소분석기 설치 위해 설치파일 복사해오기 
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

     |████████████████████████████████| 46.9 MB 46 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595735 sha256=050c7dce8e8e12392f57ca571daaf481d87cbabc88210adc60edef9cb836e1b1
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 754 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 3.0 MB 35.4 MB/s 


In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-sz4ppic3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-sz4ppic3
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=14bc40ebbe247e483fa17ed0a921bd7be1db179e9d21e51c4eb7e37a98fd9f8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-__i12xug/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [7]:
# gpu 쓸지 cpu 쓸지 확인 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
# 학습용 데이터셋 불러오기 (sop dataset 읽어옴)
input_file_name = '/content/drive/MyDrive/MLDL_telcobiz_embedding_classifier/embedding_data/sop_dataset.xlsx'
try:
    df = pd.read_excel(input_file_name, sheet_name=0, engine='openpyxl')
except FileNotFoundError:
    print(f'{input_file_name}이 없습니다! skip!')

In [10]:
# 데이터 형태 확인 
df[:5]

,담당BA부서,date,co,sentence,label_org,label_clean
0,고객채널Unit,19/05/28,SK텔레콤,[Swing]고객상담]_상담예약 한적없으나 SMS 수신 되어 문의 . [Swing]...,SWING CTC-SKT,SWING CTC
1,Billing Unit,19/05/28,SK텔레콤,[Swing]파트너정보관리]_대리점관계설정 화면에서 대리점방판코드 부여 문의 . 대...,SWING 파트너관리(PRM),SWING 파트너관리(PRM)
2,Billing Unit,19/05/28,SK텔레콤,[수납]청구요금]_철회후 재가입 한 고객 요금 환불처리 불가하여 문의 . 철회후 재...,SWING 수납,SWING Payment
3,고객상품Unit,19/05/28,SK브로드밴드,[CC]변경]_기개통된 기업전화의 기술방식변경이 가능한지 문의 . [CC]변경]_기...,SWING 유선오더,SWING 유선오더
4,고객상품Unit,19/05/28,SK텔레콤,[공통]기준정보]_체류만료일 입력대상이 아닌데 서비스신규가입화면에서 청구정보저장 누...,SWING오더 - 고객/청구정보/정지/통계,SWING 오더 - 무선오더


In [11]:
new_data = df[["sentence", "label_clean"]]

In [12]:
# sop_dataset에서 sentence와 label_clean 컬럼 사용 (X : sentence, y : label_clean)
X = df.pop('sentence')
y = df.pop('label_clean')

In [13]:
# y값 labeling (labelEncoder로 labeling 적용)
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [14]:
# categorical value로 변환되었는지 확인
print( y[:20] )
print( label_encoder.inverse_transform(y[:20]) )

[ 3 24  6 19 16 16 32 15  6 32 12 18 15 16 16 10 31 19  7 32]
['SWING CTC' 'SWING 파트너관리(PRM)' 'SWING Payment' 'SWING 유선오더'
 'SWING 오더 - 무선오더' 'SWING 오더 - 무선오더' '상품-무선' 'SWING 시설' 'SWING Payment'
 '상품-무선' 'SWING 모바일' 'SWING 유선상품' 'SWING 시설' 'SWING 오더 - 무선오더'
 'SWING 오더 - 무선오더' 'SWING 단말' '상품-단말기' 'SWING 유선오더' 'SWING Portal' '상품-무선']


In [15]:
split_corpus_X = X

In [16]:
import re

# 특수 문자열 정의
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●, ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, 【,】, …, ◆,%"
EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [17]:
!pip install hanja

     |████████████████████████████████| 120 kB 5.4 MB/s 
     |████████████████████████████████| 265 kB 25.3 MB/s 
     |████████████████████████████████| 242 kB 39.9 MB/s 
     |████████████████████████████████| 280 kB 39.4 MB/s 
     |████████████████████████████████| 242 kB 39.9 MB/s 
     |████████████████████████████████| 242 kB 38.2 MB/s 
     |████████████████████████████████| 229 kB 38.0 MB/s 
     |████████████████████████████████| 229 kB 41.7 MB/s 
INFO: pip is looking at multiple versions of pytest-cov to determine which version is compatible with other requirements. This could take a while.
  Created wheel for hanja: filename=hanja-0.13.3-py3-none-any.whl size=128423 sha256=9c75a5a358b7983b8bcfc48aafb17a36c5065fd1c7d25077caae5e191e922be6
  Stored in directory: /root/.cache/pip/wheels/70/08/88/f9cd32ddb92f5c3061cf16f068c842dc558d2f66a9c943b51a
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp37-cp37m-linux_x86_64.whl size=44117 sha256=b52faf742ad51c8c4497cd6e517e509cd581f

In [18]:
# 3. 특수 문자 제거 함수 정의
import hanja 


def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = ' '.join(sentence.split())  
    
    return sentence

def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', ' ', sentence)
    sentence = re.sub('NUM\s+', " ", sentence)
    sentence = re.sub('[NUM]+', " ", sentence)
    sentence = ' '.join(sentence.split())  

    return sentence

def cleansing_other(sentence: str = None) -> str:
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    sentence = ' '.join(sentence.split())  
        
    return sentence

def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')
    sentence = ' '.join(sentence.split())  
    
    return sentence
    
def cleansing_english(sentence: str = None) -> str:
    """
    영어 전처리 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[^가-힣0-9\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    sentence = ' '.join(sentence.split())  
    
    return sentence

def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    #sent_clean = cleansing_english(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)  

    return sent_clean

In [19]:
# 2) 특수문자, 영어, 한자, 숫자, 이메일 등 제거 시작 
# split_corpus : 문장 분리된 corpus
# cleanse_corpus : 문장 분리 + 특수문자 제거한 corpus

cleanse_corpus_X = []

for i, sentences in enumerate(split_corpus_X) :
     if i == 0 : 
        print("특수 문자 제거 작업 시작 ---------------")
     if i % 10000 == 0 : 
        print(f'{i+1}번째 문장 수행 중')   
     try :   
        preprocessed_sentences = preprocess_sent(sentences)
        if preprocessed_sentences is not None and len(preprocessed_sentences.strip()) > 1 : # 문자열 길이가 1 이상인 데이터만 담기         
          cleanse_corpus_X.append(preprocessed_sentences) 
          
     except : 
        print(f'{i+1}번째 데이터에 문제가 있어 skip!')
        print(sentence)
        continue
    
print("첫번째 데이터 확인 : ",cleanse_corpus_X[0] )

특수 문자 제거 작업 시작 ---------------
1번째 문장 수행 중
10001번째 문장 수행 중
20001번째 문장 수행 중
30001번째 문장 수행 중
40001번째 문장 수행 중
50001번째 문장 수행 중
60001번째 문장 수행 중
첫번째 데이터 확인 :  Swing 고객상담 상담예약 한적없으나 S S 수신 되어 문의 Swing 고객상담 상담예약 한적없으나 S S 수신 되어 문의 통합접척이력 조회 S S S 수신 SKT 시 상담예약 완료 으로 전화드리겠습니다 S S 수신되어 확인 부탁 드립니다 해당 서비스 이용 고객은 따로 상담 예약 하신적이 없으나 문자 발송되었다고 하여 확인 요청 합니다 요청자 최 문경


In [20]:
# 3) labeling된 y값 붙이기 
data_list = []
for sentence, label in zip(cleanse_corpus_X, y) :
  data = []
  data.append(sentence)
  data.append(str(label))
  data_list.append(data)

In [21]:
# 잘 합쳐졌나 확인
for i, data in enumerate( data_list ) : 
  if i < 5 : 
    print(data)

['Swing 고객상담 상담예약 한적없으나 S S 수신 되어 문의 Swing 고객상담 상담예약 한적없으나 S S 수신 되어 문의 통합접척이력 조회 S S S 수신 SKT 시 상담예약 완료 으로 전화드리겠습니다 S S 수신되어 확인 부탁 드립니다 해당 서비스 이용 고객은 따로 상담 예약 하신적이 없으나 문자 발송되었다고 하여 확인 요청 합니다 요청자 최 문경', '3']
['Swing 파트너정보관리 대리점관계설정 화면에서 대리점방판코드 부여 문의 대리점관계설정 화면에서 대리점방판코드 부여 문의 대리점 방문판매코드 부여하고 싶은데 방법을몰라 확인부탁드립니다 요청자 서효경', '24']
['수납 청구요금 철회후 재가입 한 고객 요금 환불처리 불가하여 문의 철회후 재가입 한 고객 요금 환불처리 불가하여 문의 철회후 재가입 한 고객 선택약정 원복 하기위해 해지번호에서 요금 수납 된 금액 환불하려고 하면 수납후 일이내 타서비스의 업무변경 가입 명변 이력으로 환불처리를 할 수 없습니다 라고 발생되고 환불처리가 불가한 상태입니다 일자에 후불 선불 로 번호이동 해지 되었던 고객 기존 선택약정 원복하기위해서는 요금에 있는 선택약정 반환금 이 환불처리 되어야 하는데 환불처리 불가하여 확인요청 문의 확인부탁드립니다 요청자', '6']
['CC 변경 기개통된 기업전화의 기술방식변경이 가능한지 문의 CC 변경 기개통된 기업전화의 기술방식변경이 가능한지 문의 화면명 기업서비스변경 서비스번호 장애현상 기개통된 기업전화의 기술방식변경이 가능한지 문의 기업 전화SOHO 기업 I S실선 기업 전화 가상중계선 해당 기술방식으로 변경할 수 있는지 확인 부탁드립니다 문의자 임성진', '19']
['공통 기준정보 체류만료일 입력대상이 아닌데 서비스신규가입화면에서 청구정보저장 누르면 체류기간 오류입니다 고객정보 체류기간변경후 처리 팝업되어 문의 공통 기준정보 체류만료일 입력대상이 아닌데 서비스신규가입화면에서 청구정보저장 누르면 체류기간 오류입니다 고객정보 체류기간변경후 처리 팝업되어 문의 화면명 서비스

In [22]:
# Train / Test set 분리
train, test = train_test_split(data_list, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 52508
test shape is: 13127


In [23]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [24]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
# Setting parameters
max_len = 100 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3 # epoch 3으로 수정
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [26]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [27]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [28]:
class SOPClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(label_encoder.classes_), # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(SOPClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
model = SOPClassifier(bertmodel, dr_rate=0.5).to(device)

In [30]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


In [31]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능


In [32]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [33]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [34]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [35]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/821 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.788438081741333 train acc 0.03125
epoch 1 batch id 201 loss 1.7987667322158813 train acc 0.2829601990049751
epoch 1 batch id 401 loss 0.7246435284614563 train acc 0.49438902743142144
epoch 1 batch id 601 loss 0.6608355641365051 train acc 0.5868084442595674
epoch 1 batch id 801 loss 0.32232582569122314 train acc 0.638713327091136
epoch 1 train acc 0.6421448146859231


  0%|          | 0/821 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.700469970703125 train acc 0.8125
epoch 2 batch id 201 loss 0.5779666900634766 train acc 0.8280472636815921
epoch 2 batch id 401 loss 0.706569492816925 train acc 0.8306966957605985
epoch 2 batch id 601 loss 0.3759947121143341 train acc 0.8336366472545758
epoch 2 batch id 801 loss 0.5235149264335632 train acc 0.8349133895131086
epoch 2 train acc 0.8351204976509483


  0%|          | 0/821 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.7830538749694824 train acc 0.765625
epoch 3 batch id 201 loss 0.6145315170288086 train acc 0.8776430348258707
epoch 3 batch id 401 loss 0.3632870614528656 train acc 0.8789744389027432
epoch 3 batch id 601 loss 0.36537930369377136 train acc 0.8818895590682196
epoch 3 batch id 801 loss 0.46211865544319153 train acc 0.8829783083645443
epoch 3 train acc 0.8829443622759701


In [36]:
# 테스트 문장 예측
test_sentence = '이번달 청구금액이 이상해요.'
test_label = 23 # 실제 label


In [37]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['sentence', 'label_clean']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [38]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-2.0664,  0.1274, -2.3140,  1.4633,  0.9538, -2.1692,  2.1637, -2.1768,
         -2.6963, -1.9016, -1.2709,  0.1072, -3.8061,  1.2293, -0.3464, -0.9338,
          0.6948, -3.0712,  0.9381,  0.0918, -2.9255, -2.3890, -0.2529,  7.2925,
         -1.2853, -1.3175, -0.9345,  0.6186,  1.8186, -0.9575, -0.3035, -0.7643,
          3.8162, -1.6113, -0.3756,  0.4619, -3.7390]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [39]:

model.eval() # 평가 모드로 변경
    
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

    loss = loss_fn(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))  
print("epoch {} test loss {}".format(e+1, loss.data.cpu().numpy()))
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/206 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.469950795173645 train acc 724.8973214285714
epoch 3 batch id 201 loss 0.6806265711784363 train acc 3.6064543354655294
epoch 3 test acc 0.8444218099861304


In [42]:
loss.data.cpu().numpy()

array(0.35957035, dtype=float32)